EITN Spring School in Computational Neuroscience, March 4-13, 2020.   
European Institute for Theoretical Neuroscience, Paris, France.


# Tutorial: Introduction to Brain Connectivity
#### Gorka Zamora-López, Ph.D.

<img src="Images/UPFlogo_smaller.png" style="float:right; width:149px; height:50px"  alt="UPF logo" />
<img src="Images/HBPlogo_smaller.png" style="float:left; width:254px; heigth:50px " alt="HBP logo" />

<br>
<br>
<br>
<br>

---------------------------

#### Dependencies

This tutorial requires a Python 3.X installation together with NumPy and Matplotlib libraries. To analyse networks we will use the *Graph Analysis Library* (**GAlib**), developed by Gorka Zamora-López. Find the library and instructions in its [GitHub repository](https://github.com/gorkazl/pyGAlib). GAlib is registered in the Python Package Index (PyPI), therefore installation only requires to run the following command in a terminal 

    $ pip install galib



### Import the main libraries

To get started, we first need to load the libraries we will use. Note that here we will make an absolute import of *NumPy*.

In [ ]:
from timeit import default_timer as timer
import matplotlib.pyplot as plt
%matplotlib inline
from numpy import*

**Import GAlib**

The following cell will check whether GAlib is installed in the current working environment, otherwise, it will call `pip` for the installation (a working internet connection is required). 

**NOTE**: The exclamation mark ( ! ) before ``pip install galib`` indicates to the Jupyter Notebook that the current line is a system command. The line is treated as if it were run from a terminal.

In [ ]:
## Check if GAlib is installed in the current environment, otherwise install
try:
    import galib
except:
    print('Installing GAlib...')
    ! pip install galib
    import galib

<br>
<br>

## Structural Connectivity ––––––––––––––––––––––––––––––––––––––––––––
#### Load the data and explore basic features of the network

In this tutorial, we will study the anatomical long-range brain connectivity in the brain of cats. The data consist of a parcellation (division) of one brain hemisphere into **53 cortical regions** (grey matter), and the anatomical long-range fibers experimentally identified between them. Find a comprehensive analysis of this network in:

- G. Zamora-López, C.S. Zhou et al "*[Cortical hubs form a module for multisensory integration on top of the hierarchy of cortical networks](https://www.frontiersin.org/articles/10.3389/neuro.11.001.2010/full)*" Front. Neuroinform. 4:1 (2010).
- G. Zamora-López, C.S. Zhou et al "*[Exploring brain function from anatomical connectivity](https://www.frontiersin.org/articles/10.3389/fnins.2011.00083/full)*" Front. Neurosci. 5:83 (2011).


Let's load the data and visualise the adjacency matrix.

In [ ]:
dataroot = 'Datasets/Cat/'
catnet = loadtxt(dataroot + 'Cat53_cortex.txt', dtype=float)

plt.figure()
plt.imshow(catnet, cmap='gray_r')
plt.colorbar()

Most graph measures exist for directed networks, which provides richer information. Check for example the `Reciprocity()` and `ReciprocalDegree()` functions. GAlib functions usually come with an optional parameter `directed` in order to specify whether the undirected or the directed version of the metric is desired. By default, the parameter is set to `directed=False`.

For the rest of the tutorial we will consider that the long-range connectome is **symmetric**, which is a fairly good approximation, and we will ignore the weights of the links.

In [ ]:
# Symmetrise and binarise the adjacency matrix
netsym = 0.5 * (catnet + catnet.T)
netsym = where(netsym,1,0).astype(uint8)

plt.figure()
plt.imshow(netsym, cmap='gray_r')
plt.colorbar()

In [ ]:
# Fundamental properties of the network
N = len(netsym)
L = 0.5 * netsym.astype(bool).sum()
Lmax = 0.5 * N*(N-1)
dens = L / Lmax

print('Number of nodes, N: %d\t\tNumber of links, L: %d\t\tdensity: %1.3f' %(N, L, dens))

<br>

### (Exercise 1) – Local network properties
Let's start analysing the network. First, we will:
1. find its degree distribution, and
2. calculate the clustering coefficient

First of all, we need to import the functions from the Graph Analysis Library (GAlib). For now, we will use the module *galib.py*, which contains functions to estimate graph measures out of a graph.

In [ ]:
# Calculate the degree of every node
deg = galib.Degree(netsym, directed=False)
print('Degrees: Min = %d, Max = %d, Mean degree: %1.3f' %(deg.min(), deg.max(), deg.mean()))

# Plot the degree distribution
plt.figure()
plt.hist(deg, bins=8, range=(0,40), rwidth=0.8, density=True)
plt.xlabel('Node degree', fontsize=14)

In [ ]:
# Calculate the clustering of the network and the local clustering of every node
C, Cnodes = galib.Clustering(netsym, checkdirected=False)

print( 'Clustering coefficient: %1.3f' %C )
print( 'Local clustering: Min = %1.3f, Max = %1.3f, Average = %1.3f' %(Cnodes.min(), Cnodes.max(), Cnodes.mean()) )

<br>

### (Exercise 2) – Rich-club identification
We say a network has a rich when (i) there are hubs in a network and (ii) those hubs are densely interconnected. Rich-club identification is a two steps process:
1. Compute the $k$-density measure, $\Phi(k)$, and see whether it grows, stabilises or decays.
2. If $k$-density grows sufficiently, identify the set of hubs forming the rich-club.

**WARNING!** Unfortunately the identification of a rich-club implies some arbitrary choices. No strict criteria exist but, as a rule of thumb, if $\Phi(k)$ grows above 0.8, then you are pretty safe. Contrary to what is commonly found in the literature, the identification of a rich-club **does not** require the comparison to results from a null-model (random or rewired networks). Such comparisons are important, however, to inquiry about the *causes* leading to the presence (or the absence) of a rich-club.

In [ ]:
# Compute the k-density and plot it
kdens = galib.k_Density(netsym)
plt.figure()
plt.plot(kdens, '.-')
plt.xlabel('Node degree', fontsize=12)
plt.ylabel('k-density', fontsize=12)
plt.grid(axis='y')

In [ ]:
# Identify the degree at which k-density overcomes the (arbitrary) threshold
densthres = 0.8
kthres = where(kdens >= densthres)[0][0]
# Identify the hubs forming the rich-club
richclub = where(deg >= kthres)[0]
print('Nodes in the richclub for k-density threshold %1.2f:' %densthres)
print(richclub)

In [ ]:
# Let's now see who those rich-club regions are
from galib.tools import LoadLabels
labels = LoadLabels(dataroot + 'Areas53_list.txt')
for node in richclub:
    print(labels[node])

<br>

### (Exercise 3) – Global network properties 

1. Compute the diameter of the network and its average pathlength
2. Calculate the betwenness centrality of the nodes

The [Dijkstra algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) is a famous method to find the graph distance between two nodes in a graph. However, if we want to know the distance between all pairs, the [Floyd-Warshall algorithm](https://en.wikipedia.org/wiki/Floyd–Warshall_algorithm) is faster than running Dijkstra N^2 times. 

In [ ]:
# Calculate the pair-wise graph distance matrix
Dij = galib.FloydWarshall(netsym)

diam = Dij.max()
avpathlen = (Dij.sum() - Dij.trace()) / (N*(N-1))

print('Diameter: %d\tAv. Pathlen: %1.3f' %(diam, avpathlen))

# Visualise the distance matrix as well

In [ ]:
# Calculate the pairwise distance matrix, the between centrality of every node,
# all shortest paths from node i to j, and all shortest path starting 
# from i and returning to i
Dij, bcnodes, paths, cycles = galib.PathsAllinOne(netsym)
print( '\nBetweenness centrality. Min: %d\tMax: %d' %(bcnodes.min(), bcnodes.max()) )

In [ ]:
# Normalise BC of the nodes by the number of existing shortest paths
# diameter was diam=3
npaths = len(paths[1]) + len(paths[2]) + len(paths[3])
bcnormed = bcnodes / npaths 

print( 'Total number of shortest paths: %d' %npaths )
print( 'Betweenness centrality. Min: %1.3f\tMax: %1.3f' %(bcnormed.min(), bcnormed.max()) )

In [ ]:
# Plot the relation between the degree and betweenness of each node
plt.figure()
plt.scatter(deg, bcnormed)
plt.xlabel('Node degree', fontsize=14)
plt.ylabel('Betweenness centrality', fontsize=14)
plt.xlim(0,45)
plt.grid(lw=0.5)

<br>

### (Exercise 4) – Graph models. Null-models and benchmarks

We will now compare the properties of the cat connectome to the properties of typical reference networks: *random graphs* and *degree-preserving random graphs*. These comparisons serve us to explore where the properties observed in real connectome may come from.

**Random graphs** are the 1st-order reference network model. It is a well-known fact that the numerical outcome of all graph metrics strongly depends on how sparse, or how dense the network is (i.e., on the size N and number of links L.) This makes network comparison a difficult challenge. Random graphs are the network models with the least number of constraints (N and L) and only one assumption: that every link has been seeded independently, at random.

Beyond N and L, we often want to know how much do the observed metrics depend on a third constraint: the degree-distribution. Specially because the presence of hubs may strongly influence other properties. To explore that, we can generate reference graphs with the same N, L and degree-sequence as the original network, which are maximally random under those constraints. Because of the additional constraint, these are 2nd-order reference graphs. We can construct such networks by randomly rewiring the links of the original network, carefully conserving the given constraints.

In GAlib library, the functions to generate different network models are located in the *models.py* module.

In [ ]:
# Generate ensemble of random graphs with same N and L as the cat connectome
from galib.models import RandomGraph
nrealiz = 10
randnets = zeros((nrealiz,N,N), uint8)
for re in range(nrealiz):
    randnets[re] = RandomGraph(N,L, directed=False)

In [ ]:
# Generate ensemble of random graphs with same N and L and k-sequence as the cat connectome
from galib.models import RewireNetwork
rewnets = zeros((nrealiz,N,N), uint8)
for re in range(nrealiz):
    rewnets[re] = RewireNetwork(netsym, prewire=10, directed=False, weighted=False)

#### Compare the degree distributions

In [ ]:
deglist_rand = []
deglist_rew = []
for re in range(nrealiz):
    deglist_rand += galib.Degree(randnets[re], directed=False).tolist()
    deglist_rew += galib.Degree(rewnets[re], directed=False).tolist()
deglist_rand = array(deglist_rand, dtype=uint)
deglist_rew = array(deglist_rew, dtype=uint)

print('\nDEGREES')
print('Cat \t Min = %d, Max = %d,\t Mean degree: %1.3f' %(deg.min(), deg.max(), deg.mean()))
print('Random \t Min = %d, Max = %d,\t Mean degree: %1.3f' %(deglist_rand.min(), deglist_rand.max(), deglist_rand.mean()) )
print('Rew \t Min = %d, Max = %d,\t Mean degree: %1.3f' %(deglist_rew.min(), deglist_rew.max(), deglist_rew.mean()))

In [ ]:
# Plot the degree distributions
plt.figure()
plt.hist(deglist_rand, bins=8, range=(0,40), rwidth=0.8, density=True)
plt.xlabel('Node degree', fontsize=14)

plt.figure()
plt.hist(deglist_rew, bins=8, range=(0,40), rwidth=0.8, density=True)
plt.xlabel('Node degree', fontsize=14)

#### Compare the clustering coefficient 

In [ ]:
Clist_rand = zeros(nrealiz, float)
Clist_rew = zeros(nrealiz, float)
for re in range(nrealiz):
    Crand, Cnodes_rand = galib.Clustering(randnets[re])
    Clist_rand[re] = Crand
    Crew, Cnodes_rew = galib.Clustering(rewnets[re])
    Clist_rew[re] = Crew

print('\nCLUSTERING COEFFICIENT')
print('Cat:\t%1.3f \nRandom:\t%1.3f \nRew:\t%1.3f' %(C, Crand, Crew))

> **NOTE: _Absolute versus relative metrics_**
>
> In the results above, `C` is the absolute or observed value of the clustering coefficient in the cat's structural connectome. We know that the clustering is a metric bounded from 0 to 1, thus `C` will inform us on whether the observed value is large or is small. 
>
> On the other hand *Crand* and *Crew* are expectation values. Thus, we should always know "*expected, respecting to what?*" `Crand` and `Crew` depend both on the constraints and a generative hypothesis under which the null-models has been constructed upon. Thus, often, we find in the literature relative metrics defined as, e.g., `Cnorm = C / Crand` or `Cnorm = C - Crand`. These "metrics" are not graph metrics, they are relative values which inform on how far does the real observation `C` deviate from expectation, given a pre-defined hypothesis. 
>
> In the case of the cat's SC, `Cnorm = C / Crand = 1.564` and `Cnorm = C / Crew = 1.230`, meaning that its clustering coefficient may be "better explained" considering the constraints imposed by the degree distribution than by the number of links alone. However, those relative values do not inform on how small or how large is the clustering coefficient of the cat's SC per se.

#### Compare the pathlength and betweenness centrality

In [ ]:
# First, calculate pathlengths and BC from the RANDOM graphs
avlenlist_rand = zeros(nrealiz, float)
bclist_rand = []
triuidx = triu_indices(N, k=1)
for re in range(nrealiz):
    Dij_rand, BC_rand, paths, cycles = galib.PathsAllinOne(randnets[re])
    values = Dij_rand[triuidx]
    avlenlist_rand[re] = values.mean()
    bclist_rand += BC_rand.tolist()

# Calculate pathlengths and BC from the REWIRED graphs
avlenlist_rew = zeros(nrealiz, float)
bclist_rew = []
for re in range(nrealiz):
    Dij_rew, BC_rew, paths, cycles = galib.PathsAllinOne(rewnets[re])
    values = Dij_rew[triuidx]
    avlenlist_rew[re] = values.mean()
    bclist_rew += BC_rew.tolist()


In [ ]:
# Results for the average pathlength 
plt.figure()
plt.title('Comparison of average pathlengths', fontsize=14)
plt.bar( (1,2,3), (avpathlen, avlenlist_rand.mean(), avlenlist_rew.mean()) )
plt.ylim(1,2)
plt.xticks((1,2,3), ('Cat', 'Random', 'Rewired') )
plt.grid(axis='y', lw=0.5)


In [ ]:
# Results for the betweenness centrality
plt.figure()
plt.scatter(deg, bcnodes)
plt.plot(deglist_rand, bclist_rand, '.', color='red', zorder=10)
plt.plot(deglist_rew, bclist_rew, '.', color='orange', zorder=5)
#plt.scatter(deg*deg, bcnodes)
#plt.plot(deglist_rand**2, bclist_rand, '.', color='red', zorder=10)
#plt.plot(deglist_rew**2, bclist_rew, '.', color='orange', zorder=5)

plt.xlabel('Node degree', fontsize=14)
plt.ylabel('Betweenness centrality', fontsize=14)
plt.grid(lw=0.5)

# Try again with k**2. (comment'xlim' out). What happens?

<br/>
<br/>

## Functional Connectivity –––––––––––––––––––––––––––––––––––––––––––––––

Usually, functional connectivity is estimated from dynamic / statistic measures from the pairwise  temporal activities of two brain regions. Here, we don't have fMRI or EEG data from the brain of cats so, for illustration purposes, we will compute an analytical estimate (a prediction) of its FC. This estimate, *Topological Similarity*, is based on a diffusion process along the network. The details of the estimate are found in:

- Bettinardi, Deco et al. (2017) "*[How structure sculpts function: Unveiling the contribution of anatomical connectivity to the brain's spontaneous correlation structure](http://aip.scitation.org/doi/10.1063/1.4980099)*" Chaos, 27, 047409.

The estimate accepts a tunable parameter, a uniform coupling strength $g$ applied to all links.

In [ ]:
# Import the function to estimate Topological Similarity
from galib.extra import TopologicalSimilarity

In [ ]:
# Coupling strength
g = 0.1  #0.0766
fcnet = TopologicalSimilarity(netsym, g)

plt.figure()
plt.imshow(fcnet, cmap='jet')
plt.clim(0,1)
plt.colorbar()

In [ ]:
# Calculate the intensity of the nodes (weighted equivalent of degree)
intens = galib.Intensity(fcnet, directed=False)

# Compare SC degree and FC intensity
plt.figure()
plt.scatter(deg, intens)
plt.xlabel('Node degree (SC)', fontsize=14)
plt.ylabel('Node intensity (FC)', fontsize=14)
plt.grid(lw=0.5)

In [ ]:
# Compare the graph distance and the pair-wise correlations

plt.figure()
for dist in range(1,int(diam)+1):
    # Get only the FC values for each graph distance...
    idx = where(Dij==dist)
    values = fcnet[idx]
    # ... and plot them
    plt.violinplot(values, positions=[dist])

plt.xlabel('Graph distance (SC)', fontsize=14)
plt.ylabel('Correlation (FC)', fontsize=14)
